In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import sys, os
# This is not super pretty, but I think this is the best way to import stuff from ../../../util?
CODE_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if CODE_ROOT not in sys.path:
    sys.path.insert(1, CODE_ROOT)

from dataset_functions import get_sdss_file_ids, get_sdss_spectra

In [6]:
cat_file = "../SDSS_support_files/BOSSLyaDR9_cat.fits"

pmf_list = get_sdss_file_ids(cat_file, 0)
print(len(pmf_list))

54468


In [7]:
resid_file = "../SDSS_support_files/residcorr_v5_4_45.dat"

# Note: this function might take a couple of minutes to run as it has to open thousands of data files
snrs, wavelengths_boss_specs, fluxes_boss_specs, pmfs_used = get_sdss_spectra(resid_file, pmf_list, return_pmf_list=True)

  0%|          | 0/54468 [00:00<?, ?it/s]/vera/u/jerbo/CLIMB-Project/code/Machine_Learning/Transformer_pipeline/dataset_functions.py:213: RuntimeWarning: divide by zero encountered in divide
  F[good_pixel] = (flux[good_pixel] * dla_corr[good_pixel] / ((cont[good_pixel] * resid[good_pixel])))
100%|██████████| 54468/54468 [28:31<00:00, 31.82it/s]


In [8]:
print(len(snrs), len(wavelengths_boss_specs), len(fluxes_boss_specs), len(pmfs_used))

24268 24268 24268 24268


In [9]:
np_specs = []
ivar_snrs = []
pmfs_used_filtered = []
for i, spec in enumerate(fluxes_boss_specs):
    if len(spec) == 402:
        np_specs.append(np.append(spec, spec[-1]))
        ivar_snrs.append(np.append(snrs[i], snrs[i][-1]))
        pmfs_used_filtered.append(pmfs_used[i])

specs = np.array(np_specs)
snrs_pp = np.array(ivar_snrs)
pmfs_specs = np.array(pmfs_used_filtered)

print(specs.shape)
print(snrs_pp.shape)
print(pmfs_specs.shape)

(24145, 403)
(24145, 403)
(24145, 3)


In [13]:
snr_medians = np.median(snrs_pp, axis=1)

In [15]:
np.savez("../SDSS_support_files/Custom_cat.npz", SNR=snr_medians, PMF=pmfs_specs)

In [16]:
# Load
data = np.load("../SDSS_support_files/Custom_cat.npz")
arr1_loaded = data["SNR"]
arr2_loaded = data["PMF"]